Perform GAN on 1D array

In [40]:
# test on occurence dataset
import numpy as np

# X = np.loadtxt('../dataset/matrix/X_occ.csv')
# X = np.loadtxt('../dataset/matrix/X_fs.csv')
# X = np.loadtxt('../dataset/matrix/X_str.csv')
# X = np.loadtxt('../dataset/matrix/X_tf_idf_sen.csv')
X = np.loadtxt('../dataset/matrix/X_tf_idf_token.csv')

Y = np.loadtxt('../dataset/matrix/Y_str.csv')


In [41]:
print(X.shape)
print(Y.shape)
from collections import Counter

Counter(Y)

(321, 125326)
(321,)


Counter({1.0: 179, 0.0: 142})

In [42]:
# build the discriminator
from keras import Sequential
from keras import layers 
from keras.callbacks import EarlyStopping

def create_model(input_dim):
    model = Sequential()
    model.add(layers.Dense(32, input_dim=input_dim, activation='relu'))
    model.add(layers.Dense(32, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

model = create_model(X.shape[1])
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 32)                4010464   
                                                                 
 dense_22 (Dense)            (None, 32)                1056      
                                                                 
 dense_23 (Dense)            (None, 1)                 33        
                                                                 
Total params: 4,011,553
Trainable params: 4,011,553
Non-trainable params: 0
_________________________________________________________________


In [43]:
from numpy.random import rand

# define the function to generate fake examples
def gen_fake_samples_alone(dim, examples):
    X = np.zeros((examples, dim))
    for i in range(examples):
        X[i] = rand(dim)
    
    Y = np.zeros((examples,1))

    return np.array(X), Y

In [44]:
# train the discriminator on both real and fake examples
def train_cls(epochs, batches, max, model, real_x, real_y):
    half_batch = int(batches/2)

    for i in range(epochs):
        if i*half_batch < max:
            # update the model with real dataset
            model.train_on_batch(real_x[half_batch*i:half_batch*(i+1)], real_y[half_batch*i:half_batch*(i+1)])

            # generate fake examples
            X_fake, Y_fake = gen_fake_samples_alone(real_x.shape[1], half_batch)
            # update on fake datasets
            model.train_on_batch(X_fake, Y_fake)
            
            _, acc_real = model.evaluate(real_x, real_y, verbose=0)
            
            _, acc_fake = model.evaluate(X_fake, Y_fake, verbose=0)
            
            print(i, acc_real, acc_fake)



In [46]:
# fit the model
train_cls(1000, 10, 179, model, X, Y)

0 0.5576323866844177 1.0
1 0.5576323866844177 1.0
2 0.5576323866844177 1.0
3 0.5576323866844177 1.0
4 0.5576323866844177 1.0
5 0.5576323866844177 1.0
6 0.5576323866844177 1.0
7 0.5576323866844177 1.0
8 0.5576323866844177 1.0
9 0.5576323866844177 1.0
10 0.5576323866844177 1.0
11 0.5576323866844177 1.0
12 0.5576323866844177 1.0
13 0.5576323866844177 1.0
14 0.5576323866844177 1.0
15 0.5576323866844177 1.0
16 0.5576323866844177 1.0
17 0.5576323866844177 1.0
18 0.5576323866844177 1.0
19 0.5576323866844177 1.0
20 0.5576323866844177 1.0
21 0.5576323866844177 1.0
22 0.5576323866844177 1.0
23 0.5576323866844177 1.0
24 0.5576323866844177 1.0
25 0.5576323866844177 1.0
26 0.5576323866844177 1.0
27 0.5576323866844177 1.0
28 0.5576323866844177 1.0
29 0.5576323866844177 1.0
30 0.5576323866844177 1.0
31 0.5576323866844177 1.0
32 0.5576323866844177 1.0
33 0.5576323866844177 1.0
34 0.5576323866844177 1.0
35 0.5576323866844177 1.0


Build the basic GAN Model

In [58]:
def gen_generator(n_outputs, n_latent):
    model = Sequential()
    model.add(layers.Dense(32, input_dim=n_latent, kernel_initializer='he_uniform', activation='relu'))
    model.add(layers.Dense(n_outputs, activation='linear'))
    print('Summary for generator model: ', model.summary())
    
    return model

# build standalone discriminator
def gen_discriminator(n_inputs):
    model = Sequential()
    model.add(layers.Dense(32, input_dim = n_inputs, activation='relu', kernel_initializer='he_uniform'))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    print('Summary for discriminator model: ',model.summary())

    return model

def gen_gan(generator, discriminator):
    # set the trainiable as False
    discriminator.trainable=False
    
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    model.compile(loss='binary_crossentropy', optimizer='adam')

    return model

In [63]:
# build the generator model

def gen_latent_points(latent_dim, n):
    x = rand(latent_dim * n)
    x.reshape((n, latent_dim))

    return x

def gen_fake_samples(generator, latent_dim, n):
    inputs = gen_latent_points(latent_dim, n)
    X = generator.predict(inputs)
    print(X)
    return X

In [68]:
g_model = gen_generator(X.shape[1], 5)
gen_fake_samples(g_model, 5, 5)

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_56 (Dense)            (None, 32)                192       
                                                                 
 dense_57 (Dense)            (None, 125326)            4135758   
                                                                 
Total params: 4,135,950
Trainable params: 4,135,950
Non-trainable params: 0
_________________________________________________________________
Summary for generator model:  None


ValueError: in user code:

    File "C:\Users\ren\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\ren\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\ren\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\ren\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1751, in predict_step
        return self(x, training=False)
    File "C:\Users\ren\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\ren\AppData\Lo

In [64]:
# train the generator

def train_gan(gan_model, latent_dim, n_epochs=10000, n_batch=128):
	# manually enumerate epochs
	for i in range(n_epochs):
		# prepare points in latent space as input for the generator
		x_gan = gen_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		y_gan = np.zeros((n_batch, 1))
		# update the generator via the discriminator's error
		gan_model.train_on_batch(x_gan, y_gan)


def train_gan_com(n_epochs, n_batch, g_model, d_model, gan_model, latent_dim, real_x):
	# define the batches
	half_batch = int(n_batch//2)
	# train with the epochs
	for i in range(n_epochs):
		# genreate fake examples by generator
		fake_x = gen_fake_samples(g_model, latent_dim, half_batch)
		# 1 is the real malware class, 0 is the fake class
		fake_y = np.zeros((half_batch, 1))
		real_y = np.ones(half_batch, 1)
		# update distriminator with both real and fake examples
		d_model.train_on_batch(real_x[half_batch*i:half_batch*(i+1)], real_y[half_batch*i:half_batch*(i+1)])
		d_model.train_on_batch(fake_x, fake_y)
		# prepare points in latent space as input for the generator
		x_gan = gen_latent_points(latent_dim, n_batch)
		# create inverted labels for the fake samples
		# update the generator via the discriminator's error
		gan_model.train_on_batch(x_gan, fake_y)


In [67]:
latent_dim = X.shape[1]
g_model = gen_generator(latent_dim, latent_dim)
d_model = gen_discriminator(latent_dim)
gan_model = gen_gan(g_model, d_model)
train_gan_com(1000, 10, g_model, d_model, gan_model, latent_dim, X)

Model: "sequential_29"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_52 (Dense)            (None, 32)                4010464   
                                                                 
 dense_53 (Dense)            (None, 125326)            4135758   
                                                                 
Total params: 8,146,222
Trainable params: 8,146,222
Non-trainable params: 0
_________________________________________________________________
Summary for generator model:  None
Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_54 (Dense)            (None, 32)                4010464   
                                                                 
 dense_55 (Dense)            (None, 1)                 33        
                                                   

ValueError: in user code:

    File "C:\Users\ren\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\ren\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\ren\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\ren\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\engine\training.py", line 1751, in predict_step
        return self(x, training=False)
    File "C:\Users\ren\AppData\Local\Programs\Python\Python38\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\ren\AppData\Lo